## This Notebook demonstrates how to reduce the bias during "In-processing" stage using AI 360 Fairness toolkit

### In-processing algorithm
A bias mitigation algorithm that is applied to a model during its training.

### Insert your credentials as credentials in the below cell
Click on dropdown from Pipeline_LabelEncoder-0.1.zip under Data tab and select 'Credentials'

In [2]:
from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])

In [3]:
import os
os.getcwd()

'/home/wsuser/work'

In [4]:
cos.download_file(Bucket=credentials['BUCKET'],Key='Pipeline_LabelEncoder-0.1.zip',Filename='/home/wsuser/work/Pipeline_LabelEncoder-0.1.zip')

In [5]:
!ls

Pipeline_LabelEncoder-0.1.zip


In [6]:
!pip install Pipeline_LabelEncoder-0.1.zip
!pip install aif360
!pip install fairlearn

Processing ./Pipeline_LabelEncoder-0.1.zip
  Created wheel for Pipeline-LabelEncoder: filename=Pipeline_LabelEncoder-0.1-py3-none-any.whl size=2062 sha256=b79f7abe3b5f5b526227d7e57867bccf538ea3527b0ac0308bf298044aa59af5
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/6b/4b/1e/43f3c8b97ffade4539a329b9eaa5755e4df16a248960947534
Successfully built Pipeline-LabelEncoder
  Attempting uninstall: Pipeline-LabelEncoder
    Found existing installation: Pipeline-LabelEncoder 0.1
    Uninstalling Pipeline-LabelEncoder-0.1:
      Successfully uninstalled Pipeline-LabelEncoder-0.1


In [7]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.__version__

'2.4.4'

In [8]:
%matplotlib inline
# Load all necessary packages
import pandas as pd
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

### Insert the data as Pandas Dataframe and change the name from df_data_ to df

,Gender,Married,Education,Fraud_risk
0,Male,No,1,Risk
1,Male,Yes,1,Safe
2,Male,Yes,1,Safe
3,Male,Yes,0,Safe
4,Male,No,1,Risk


In [10]:
df.describe(include = 'all')

,Gender,Married,Education,Fraud_risk
count,921,921,921.000000,921
unique,2,2,NaN,2
top,Male,No,NaN,Safe
freq,703,501,NaN,562
mean,NaN,NaN,0.730727,NaN
std,NaN,NaN,0.443823,NaN
min,NaN,NaN,0.000000,NaN
25%,NaN,NaN,0.000000,NaN
50%,NaN,NaN,1.000000,NaN
75%,NaN,NaN,1.000000,NaN


In [11]:
privileged_groups = [{'Gender': 1}]
unprivileged_groups = [{'Gender': 0}]
favorable_label = 1 
unfavorable_label = 0

In [12]:
from sklearn import preprocessing
categorical_column = ['Gender', 'Married', 'Fraud_risk']

data_encoded = df.copy(deep=True)
#Use Scikit-learn label encoding to encode character data
lab_enc = preprocessing.LabelEncoder()
for col in categorical_column:
        data_encoded[col] = lab_enc.fit_transform(df[col])
        le_name_mapping = dict(zip(lab_enc.classes_, lab_enc.transform(lab_enc.classes_)))
        print('Feature', col)
        print('mapping', le_name_mapping)
        

data_encoded.head()

Feature Gender
mapping {'Female': 0, 'Male': 1}
Feature Married
mapping {'No': 0, 'Yes': 1}
Feature Fraud_risk
mapping {'Risk': 0, 'Safe': 1}


,Gender,Married,Education,Fraud_risk
0,1,0,1,0
1,1,1,1,1
2,1,1,1,1
3,1,1,0,1
4,1,0,1,0


In [13]:
from Pipeline_LabelEncoder.sklearn_label_encoder import PipelineLabelEncoder
preprocessed_data = PipelineLabelEncoder(columns = ['Gender','Married', 'Fraud_risk']).fit_transform(data_encoded)
print('-------------------------')
#print('validation data encoding')
#validation_enc_data = PipelineLabelEncoder(columns = ['Gender','Married', 'Fraud_risk']).transform(validation_input_data)

Inside fit transform
Feature Gender
mapping {0: 0, 1: 1}
Feature Married
mapping {0: 0, 1: 1}
Feature Fraud_risk
mapping {0: 0, 1: 1}
-------------------------


In [14]:
#Create binary label dataset that can be used by bias mitigation algorithms
fraud_dataset = BinaryLabelDataset(favorable_label=favorable_label,
                                unfavorable_label=unfavorable_label,
                                df=preprocessed_data,
                                label_names=['Fraud_risk'],
                                protected_attribute_names=['Gender', 'Married'],
                                unprivileged_protected_attributes=unprivileged_groups)

In [15]:
display(Markdown("#### Training Data Details"))
print("shape of the training dataset", fraud_dataset.features.shape)
print("Training data favorable label", fraud_dataset.favorable_label)
print("Training data unfavorable label", fraud_dataset.unfavorable_label)
print("Training data protected attribute", fraud_dataset.protected_attribute_names)
print("Training data privileged protected attribute (1:Male and 0:Female)", 
      fraud_dataset.privileged_protected_attributes)
print("Training data unprivileged protected attribute (1:Male and 0:Female)",
      fraud_dataset.unprivileged_protected_attributes)

#### Training Data Details

shape of the training dataset (921, 3)
Training data favorable label 1.0
Training data unfavorable label 0.0
Training data protected attribute ['Gender', 'Married']
Training data privileged protected attribute (1:Male and 0:Female) [array([1.]), array([1.])]
Training data unprivileged protected attribute (1:Male and 0:Female) [array([0.]), array([0.])]


In [16]:
fraud_dataset_train, fraud_dataset_test = fraud_dataset.split([0.9], shuffle=True)

In [17]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(fraud_dataset_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(fraud_dataset_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.371405
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.392308


In [18]:
min_max_scaler = MaxAbsScaler()
fraud_dataset_train.features = min_max_scaler.fit_transform(fraud_dataset_train.features)
fraud_dataset_test.features = min_max_scaler.transform(fraud_dataset_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(fraud_dataset_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(fraud_dataset_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.371405
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.392308


### Build plan classifier without debiasing

In [19]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False

sess = tf.compat.v1.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [20]:
plain_model.fit(fraud_dataset_train)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
epoch 0; iter: 0; batch classifier loss: 0.782193
epoch 1; iter: 0; batch classifier loss: 0.703004
epoch 2; iter: 0; batch classifier loss: 0.667041
epoch 3; iter: 0; batch classifier loss: 0.630537
epoch 4; iter: 0; batch classifier loss: 0.623781
epoch 5; iter: 0; batch classifier loss: 0.586897
epoch 6; iter: 0; batch classifier loss: 0.552156
epoch 7; iter: 0; batch classifier loss: 0.563670
epoch 8; iter: 0; batch classifier loss: 0.538822
epoch 9; iter: 0; batch classifier loss: 0.493203
epoch 10; iter: 0; batch classifier loss: 0.513934
epoch 11; iter: 0; batch classifier loss: 0.456346
epoch 12; iter: 0; batch classifier loss: 0.475201
epoch 13; iter: 0; batch classifier loss: 0.461812
epoch 14; iter: 0; batch classifier loss: 0.430270
epoch 15; iter: 0; batch classifier loss: 0.454768
epoch 16; iter: 0; batch classifier loss: 0.448100
epoch 17; iter: 0; batch cla

### Apply the plain model to test data

In [21]:
dataset_nodebiasing_train = plain_model.predict(fraud_dataset_train)
dataset_nodebiasing_test = plain_model.predict(fraud_dataset_test)

### Metrics for the dataset from plain model (without debiasing)

In [22]:
display(Markdown("#### Model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(fraud_dataset_test, 
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.509533
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.610577


#### Model - without debiasing - classification metrics

Test set: Classification accuracy = 0.806452
Test set: Balanced classification accuracy = 0.802273
Test set: Disparate impact = 0.111888
Test set: Equal opportunity difference = -0.607143
Test set: Average odds difference = -0.449405
Test set: Theil_index = 0.152998


### Apply in-processing algorithm based on adversarial learning

In [23]:
sess.close()
tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.Session()

In [24]:
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [25]:
debiased_model.fit(fraud_dataset_train)

epoch 0; iter: 0; batch classifier loss: 0.657903; batch adversarial loss: 0.573822
epoch 1; iter: 0; batch classifier loss: 0.649026; batch adversarial loss: 0.571995
epoch 2; iter: 0; batch classifier loss: 0.627158; batch adversarial loss: 0.565493
epoch 3; iter: 0; batch classifier loss: 0.628682; batch adversarial loss: 0.589801
epoch 4; iter: 0; batch classifier loss: 0.579517; batch adversarial loss: 0.582512
epoch 5; iter: 0; batch classifier loss: 0.585141; batch adversarial loss: 0.593441
epoch 6; iter: 0; batch classifier loss: 0.585325; batch adversarial loss: 0.628145
epoch 7; iter: 0; batch classifier loss: 0.550264; batch adversarial loss: 0.566879
epoch 8; iter: 0; batch classifier loss: 0.533809; batch adversarial loss: 0.606753
epoch 9; iter: 0; batch classifier loss: 0.566627; batch adversarial loss: 0.610721
epoch 10; iter: 0; batch classifier loss: 0.504185; batch adversarial loss: 0.574993
epoch 11; iter: 0; batch classifier loss: 0.561196; batch adversarial loss:

### Apply the plain model to test data

In [26]:
dataset_debiasing_train = debiased_model.predict(fraud_dataset_train)
dataset_debiasing_test = debiased_model.predict(fraud_dataset_test)

In [27]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(fraud_dataset_test, 
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.509533
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.610577


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.462279
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.450000


#### Model - without debiasing - classification metrics

Test set: Classification accuracy = 0.806452
Test set: Balanced classification accuracy = 0.802273
Test set: Disparate impact = 0.111888
Test set: Equal opportunity difference = -0.607143
Test set: Average odds difference = -0.449405
Test set: Theil_index = 0.152998


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.752688
Test set: Balanced classification accuracy = 0.740152
Test set: Disparate impact = 1.818182
Test set: Equal opportunity difference = 0.232143
Test set: Average odds difference = 0.595238
Test set: Theil_index = 0.186823


## We have observed how to use AI 360 fairness toolkit to eliminate the bias during preprocessing & inprocessing stages of model building & development.